In [1]:
# install.packages('gplots', repos='http://cran.us.r-project.org')
# source("https://bioconductor.org/biocLite.R")
# biocLite("vsn")
# install.packages('RColorBrewer', repos='http://cran.us.r-project.org')
# source("https://bioconductor.org/biocLite.R")
# biocLite("edgeR")

library(tidyr)
library(dplyr)
library(ggplot2)
library(RColorBrewer)
library(glmnet)
library(glmnet)
library(gplots)

# library(missForest)
# library(huge)
# library(JGL)
# library(msgl)
# library(edgeR)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: Matrix

Attaching package: ‘Matrix’

The following object is masked from ‘package:tidyr’:

    expand

Loading required package: foreach
Loaded glmnet 2.0-5


Attaching package: ‘gplots’

The following object is masked from ‘package:stats’:

    lowess



In [2]:
load("/proj/price1/CHDI/studies/allelic_series/HDinHD_FPKMs/all_mrna_data_NPN.Rdata")
load("/proj/price1/CHDI/studies/allelic_series/HDinHD_FPKMs/all_mrna_data_tidy_NPN.Rdata")
load("/proj/price1/CHDI/studies/allelic_series/HDinHD_FPKMs/all_mrna_data_FPKM.Rdata")
load("/proj/price1/CHDI/studies/allelic_series/HDinHD_FPKMs/all_mrna_data_tidy_FPKM.Rdata")
load("/proj/price1/CHDI/studies/allelic_series/HDinHD_FPKMs/all_mrna_data_RAW.Rdata")
load("/proj/price1/CHDI/studies/allelic_series/HDinHD_FPKMs/all_mrna_data_tidy_RAW.Rdata")

In [3]:
ls()

[1] "all_mrna_data_FPKM"      "all_mrna_data_FPKM_tidy"
[3] "all_mrna_data_npn"       "all_mrna_data_npn_tidy" 
[5] "all_mrna_data_raw"       "all_mrna_data_raw_tidy" 
[7] "q"                       "quit"

In [4]:
sum(is.na(all_mrna_data_npn))

[1] 40

In [5]:
dim(all_mrna_data_npn)

[1]   716 12896

In [11]:
all_mrna_data_npn <-cbind(Sample.ID=rownames(all_mrna_data_npn),all_mrna_data_npn)

In [13]:
head(all_mrna_data_npn[,1:10])

,Sample.ID,Numeric.Q,Sex,Genotyped.CAG,Tissue,Timepoint,0610005C13Rik,0610007N19Rik,0610007P14Rik,0610008F07Rik
X28210_1_641L_striatum_Q20_WT_M_L3.LB9,X28210_1_641L_striatum_Q20_WT_M_L3.LB9,20,MALE,7,cortex,10,-1.33965005372848,-1.01885354370688,-0.477390018583274,-0.332061236789903
X299971_Q80B0021_10_L4.D709,X299971_Q80B0021_10_L4.D709,80,FEMALE,85.137,cortex,10,0.223967597372717,-0.79431411467698,-0.942484732953563,-0.332061236789903
X299972_Q80B0021_06_L4.D710,X299972_Q80B0021_06_L4.D710,80,FEMALE,84.098,cortex,10,-0.426704096771064,-0.54965360658391,-0.541490511420735,-0.332061236789903
X299973_Q111B0021_06_L4.D711,X299973_Q111B0021_06_L4.D711,111,FEMALE,127.973,cortex,10,0.16111741792323,-0.29873533559768,-1.04262934740195,-0.332061236789903
X299974_Q20B0017_04_L4.D712,X299974_Q20B0017_04_L4.D712,20,MALE,20.0086,cortex,10,0.0176310948025585,-1.40179375398891,-0.907581878451486,-0.332061236789903
X299975_Q175B0591_08_L28,X299975_Q175B0591_08_L28,175,MALE,198.141,cortex,10,-0.754073718017269,-0.674763184351501,-0.392242277472293,-0.332061236789903


In [14]:
data_subset_cortex_10 <- filter(all_mrna_data_npn, 
                    Tissue == "cortex",
                    Timepoint == 6
                    )
dim(data_subset_cortex_10)

[1]    56 12897

In [17]:
data_subset_cortex_10_meta <- data_subset_cortex_10[,1:6]
dim(data_subset_cortex_10_meta)

[1] 56  6

In [16]:
data_subset_cortex_10_meta[with(data_subset_cortex_10_meta,order(Numeric.Q)),]

,Sample.ID,Numeric.Q,Sex,Genotyped.CAG,Tissue,Timepoint
3,299882_Q20B0001_05_L1,20,MALE,7,cortex,6
4,299883_Q20B0005_03_L12,20,FEMALE,7,cortex,6
5,299884_Q20B0005_05_L5.D703,20,FEMALE,7,cortex,6
6,299885_Q20B0001_03_L6.D704,20,MALE,7,cortex,6
8,299712_Q20B0001_01_L1.D705,20,MALE,7,cortex,6
11,299715_Q20B0005_05_L1.D708,20,FEMALE,7,cortex,6
15,299721_Q20B0005_03_L2.D712,20,FEMALE,7,cortex,6
16,299722_Q20B0001_07_L2.D701,20,MALE,7,cortex,6
17,299723_Q20B0001_03_L3.D702,20,MALE,7,cortex,6
19,299725_Q20B0005_07_L3.D704,20,FEMALE,7,cortex,6
